In [ ]:
# Ensure project root is on sys.path for absolute imports
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
from agents.rag_ingest import initialize_vectorstore_with_rag_chain
from pathlib import Path
from tempfile import mkdtemp

In [16]:
FILE_PATH = "https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"
TOP_K = 3
milvus_uri = str(Path(mkdtemp()) / "vector.db")
collection_name="vectordb"

In [ ]:
rag_chain = initialize_vectorstore_with_rag_chain(
    FILE_PATH=FILE_PATH,
    TOP_K=TOP_K,
    milvus_uri=milvus_uri,
    collection_name=collection_name,
)

2026-01-09 01:11:59,374 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 01:11:59,427 - INFO - Going to convert document batch...
2026-01-09 01:11:59,432 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2026-01-09 01:11:59,448 - INFO - Auto OCR model selected ocrmac.
2026-01-09 01:11:59,456 - INFO - Accelerator device: 'mps'


In [ ]:
from agents.retrieval_orchestrator_agent import create_retrieval_orchestrator_agent, Context
from agents.resoning_agent import create_reasoning_agent

In [ ]:
retrieval_orchestrator_agent = create_retrieval_orchestrator_agent()
reasoning_agent = create_reasoning_agent()

# Load reasoning prompt for dynamic formatting
def read_markdown_file(filepath):
    """Reads the content of a Markdown file as a string."""
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            text = f.read()
        return text
    except FileNotFoundError:
        return f"Error: The file at {filepath} was not found."
    except Exception as e:
        return f"An error occurred: {e}"

reasoning_prompt = read_markdown_file("../prompts/reasoning_prompt.md")

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from typing import Annotated, Any
from langchain_core.messages import HumanMessage, SystemMessage
import json

In [ ]:
class MainState(MessagesState):
    question: Annotated[str, "The user's question"]
    retrieval_results: Annotated[dict | None, "The retrieval results from the retrieval orchestrator agent"] = None
    final_answer: Annotated[str | None, "The final answer generated by the reasoning agent"] = None
    human_approval: Annotated[bool | None, "Whether the human approved the final answer"] = None
    rag_chain: Annotated[Any, "The RAG chain to use for retrieval"]
    pending_review: Annotated[Any | None, "HITL review configs returned on interrupt"] = None

In [ ]:
def invoke_retrieval_orchestration(state: MainState):
    """Invoke the retrieval orchestrator agent to get context from RAG."""
    # Get the user question from state or last message
    user_question = state.get("question") or (state.get("messages", [])[-1].content if state.get("messages") else None)
    if not user_question:
        raise ValueError("No question found in state")
    
    # Get rag_chain from state
    rag_chain = state.get("rag_chain")
    if not rag_chain:
        raise ValueError("No rag_chain found in state")
    
    result = retrieval_orchestrator_agent.invoke(
        {
            "messages": [
                {"role": "user", "content": user_question}
            ],
        },
        context=Context(rag_chain=rag_chain)
    )
    res_messages = result["messages"]
    retrieval_results = result["structured_response"]["results"]
    return {
        "messages": res_messages, 
        "retrieval_results": retrieval_results,
        "question": user_question  # Ensure question is set in state
    }

In [ ]:
def invoke_reasoning(state: MainState):
    """Invoke the reasoning agent with retrieval results."""
    # Get question and retrieval results from state
    user_question = state.get("question") or (state.get("messages", [])[-1].content if state.get("messages") else None)
    if not user_question:
        raise ValueError("No question found in state")
    
    retrieval_results = state.get("retrieval_results")
    if not retrieval_results:
        raise ValueError("No retrieval results found in state")
    
    # Format the reasoning prompt with user question and context
    formatted_prompt = reasoning_prompt.format(
        user_question=user_question, 
        context=json.dumps(retrieval_results, indent=2)
    )
    
    # Create messages with system prompt and user question
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=user_question)
    ]
    
    cfg = {"configurable": {"thread_id": "reasoning-thread"}}

    result = reasoning_agent.invoke({"messages": messages}, config=cfg)
    res_messages = result["messages"]

    # If the agent interrupted for HITL, store review configs; graph edges decide routing
    if "__interrupt__" in result:
        review_configs = result["__interrupt__"][-1].value["review_configs"]
        return {"messages": res_messages, "pending_review": review_configs}

    # Extract final answer from structured response
    final_answer = result.get("structured_response", {}).get("final_answer", "")
    if not final_answer:
        # Fallback: get from last message if structured response is missing
        final_answer = res_messages[-1].content if res_messages else ""
    
    return {
        "messages": res_messages, 
        "final_answer": final_answer, 
        "pending_review": None
    }

In [ ]:
def human_review(state: MainState):
    """Handle human review for web search approval."""
    # Present state.pending_review to a human and collect decision.
    # For now, auto-approve to demonstrate resume flow.
    # In production, you would show the pending_review to the user and wait for input
    pending_review = state.get("pending_review")
    print("Human review needed for web search approval.")
    print(f"Pending review: {pending_review}")
    
    decision = {"type": "approve"}  # Auto-approve for demo; replace with user input
    cfg = {"configurable": {"thread_id": "reasoning-thread"}}

    resumed = reasoning_agent.invoke(
        Command(resume={"decisions": [decision]}),
        config=cfg,
    )

    res_messages = resumed["messages"]
    
    # Extract final answer from structured response
    final_answer = resumed.get("structured_response", {}).get("final_answer", "")
    if not final_answer:
        # Fallback: get from last message if structured response is missing
        final_answer = res_messages[-1].content if res_messages else ""

    # Graph-directed: only update; builder edge sends us to END
    return {
        "messages": res_messages,
        "final_answer": final_answer,
        "human_approval": True,
        "pending_review": None,
    }

In [ ]:
builder = StateGraph(MainState)

builder.add_node("invoke_retrieval_orchestration", invoke_retrieval_orchestration)
builder.add_node("invoke_reasoning", invoke_reasoning)
builder.add_node("human_review", human_review)

builder.add_edge(START, "invoke_retrieval_orchestration")
builder.add_edge("invoke_retrieval_orchestration", "invoke_reasoning")

# Route based on presence of pending_review set by invoke_reasoning
def should_review(state: MainState) -> str:
    """Determine if human review is needed."""
    pending = state.get("pending_review")
    if pending is not None:
        return "human_review"
    return "end"

builder.add_conditional_edges(
    "invoke_reasoning",
    should_review,
    {
        "human_review": "human_review",
        "end": END,
    }
)

builder.add_edge("human_review", END)

graph = builder.compile()

In [ ]:
# Initialize the graph with a user question
user_question = "How does the transformer model work?"

# Initialize state with the question and rag_chain
initial_state = {
    "messages": [HumanMessage(content=user_question)],
    "question": user_question,
    "rag_chain": rag_chain,
    "retrieval_results": None,
    "final_answer": None,
    "human_approval": None,
    "pending_review": None,
}

# Invoke the graph
config = {"configurable": {"thread_id": "main-thread"}}
result = graph.invoke(initial_state, config=config)

print("=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result["final_answer"])
print("\n" + "=" * 80)